# Install & Load Packages

In [1]:
%pip install -U "sentence-transformers[train]" " transformers[torch]" accelerate datasets pandas matplotlib seaborn numpy

                                              0.0/227.1 kB ? eta -:--:--
     ------------------                     112.6/227.1 kB 3.3 MB/s eta 0:00:01
     --------------------------------       194.6/227.1 kB 2.0 MB/s eta 0:00:01
     -------------------------------------  225.3/227.1 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 227.1/227.1 kB 1.4 MB/s eta 0:00:00
                                              0.0/315.1 kB ? eta -:--:--
     -------                                 61.4/315.1 kB 1.7 MB/s eta 0:00:01
     -------------                          112.6/315.1 kB 1.3 MB/s eta 0:00:01
     ----------------------------           235.5/315.1 kB 1.3 MB/s eta 0:00:01
     -------------------------------------  307.2/315.1 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 315.1/315.1 kB 1.1 MB/s eta 0:00:00
                                              0.0/547.8 kB ? eta -:--:--
     ----                                    61.4/547.8 kB 1.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.24.4 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import libraries
from datasets import load_dataset, Dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments
)
from sentence_transformers.losses import CoSENTLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
import pandas as pd
import numpy as np

c:\Users\Haikal\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Fine-tuning Model

In [4]:
# Define model
model = SentenceTransformer("firqaaa/indo-sentence-bert-base")

# Define loss function (CoSENTLoss | Cosine Sentence Loss -> Returning float similarity score)
loss = CoSENTLoss(model)

In [6]:
# Specify training args
args = SentenceTransformerTrainingArguments(
    output_dir="fine-tuned/sbert-fine-tuned-chatPMB",
    num_train_epochs=100,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
)

In [11]:
# Creating eval dataset
train_patterns = pd.read_csv('data/preprocessed-data-v2.csv')['pattern']
test_patterns = pd.read_csv('data/dataset-question-v2.csv')['pattern']
# test_patterns = test_patterns.iloc[:-154]
print(train_patterns.shape, test_patterns.shape)

(558,) (558,)


In [12]:
embed_train = model.encode(train_patterns)
embed_test = model.encode(test_patterns)

In [13]:
scores = []
sentences_1 = []
sentences_2 = []

for i in range(len(test_patterns)):
    data = model.similarity(embed_train[int(i)], embed_test[int(i)])
    data = float(data[0][0])
    s1 = train_patterns[int(i)]
    s2 = test_patterns[int(i)]
    sentences_1.append(s1)
    sentences_2.append(s2)
    scores.append(data)

df = pd.DataFrame({
    "sentence_1": sentences_1,
    "sentence_2": sentences_2,
    "label": scores,
})
df.to_csv('data/fine-tuned-dataset.csv')
df.tail()

,sentence_1,sentence_2,label
553,program beasiswa sedia uin sunan gunung djati ...,Apakah program beasiswa disediakan oleh UIN Su...,0.903670
554,program beasiswa uin sunan gunung djati bandun...,Program beasiswa di UIN Sunan Gunung Djati Ban...,0.887859
555,beasiswa uin sunan gunung djati bandung ada,Apakah beasiswa di UIN Sunan Gunung Djati Band...,0.888745
556,program beasiswa uin sunan gunung djati bandun...,Program beasiswa di UIN Sunan Gunung Djati Ban...,0.888634
557,beasiswa uin sunan gunung djati bandung beri,Apakah beasiswa di UIN Sunan Gunung Djati Band...,0.892302


In [14]:
# Create evaluator & evaluate the base model
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=df["sentence_1"],
    sentences2=df["sentence_2"],
    scores=df["label"],
    main_similarity=SimilarityFunction.COSINE,
    show_progress_bar=True,
    precision="float32",
    name="train-evaluator",
)
dev_evaluator(model)

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Batches: 100%|██████████| 35/35 [00:23<00:00,  1.49it/s]


{'train-evaluator_pearson_cosine': 0.9999999999984652,
 'train-evaluator_spearman_cosine': 0.9999993956485574,
 'train-evaluator_pearson_manhattan': 0.9754313537197102,
 'train-evaluator_spearman_manhattan': 0.9992904394150622,
 'train-evaluator_pearson_euclidean': 0.9749967817818792,
 'train-evaluator_spearman_euclidean': 0.9999992747782398,
 'train-evaluator_pearson_dot': 0.9999999999990659,
 'train-evaluator_spearman_dot': 0.9999993956487196,
 'train-evaluator_pearson_max': 0.9999999999990659,
 'train-evaluator_spearman_max': 0.9999993956487196}

In [16]:
df.shape

(558, 3)

In [17]:
training_data = Dataset.from_dict(df)
training_data

Dataset({
    features: ['sentence_1', 'sentence_2', 'label'],
    num_rows: 558
})

In [18]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=training_data,
    # eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()

  4%|▍         | 100/2400 [12:36<4:57:37,  7.76s/it]

{'loss': 4.3011, 'grad_norm': 21.986669540405273, 'learning_rate': 8.333333333333334e-06, 'epoch': 3.79}


  4%|▍         | 100/2400 [13:25<4:57:37,  7.76s/it]

{'eval_train-evaluator_pearson_cosine': 0.9516155808784074, 'eval_train-evaluator_spearman_cosine': 0.9523163665196182, 'eval_train-evaluator_pearson_manhattan': 0.9265545754418698, 'eval_train-evaluator_spearman_manhattan': 0.9522315139076578, 'eval_train-evaluator_pearson_euclidean': 0.926455767418, 'eval_train-evaluator_spearman_euclidean': 0.952316381316748, 'eval_train-evaluator_pearson_dot': 0.9516155833327419, 'eval_train-evaluator_spearman_dot': 0.9523164750861552, 'eval_train-evaluator_pearson_max': 0.9516155833327419, 'eval_train-evaluator_spearman_max': 0.9523164750861552, 'eval_runtime': 48.8363, 'eval_samples_per_second': 0.0, 'eval_steps_per_second': 0.0, 'epoch': 3.79}


  8%|▊         | 200/2400 [25:30<5:10:27,  8.47s/it] 

...

100%|██████████| 2400/2400 [4:58:07<00:00,  5.65s/it]

{'eval_train-evaluator_pearson_cosine': 0.9275069308083301, 'eval_train-evaluator_spearman_cosine': 0.9533061052805893, 'eval_train-evaluator_pearson_manhattan': 0.8881402151123426, 'eval_train-evaluator_spearman_manhattan': 0.952331818183709, 'eval_train-evaluator_pearson_euclidean': 0.8884187837558797, 'eval_train-evaluator_spearman_euclidean': 0.9533021486755879, 'eval_train-evaluator_pearson_dot': 0.9275069262105775, 'eval_train-evaluator_spearman_dot': 0.9533056672861877, 'eval_train-evaluator_pearson_max': 0.9275069308083301, 'eval_train-evaluator_spearman_max': 0.9533061052805893, 'eval_runtime': 43.844, 'eval_samples_per_second': 0.0, 'eval_steps_per_second': 0.0, 'epoch': 89.0}


100%|██████████| 2400/2400 [4:58:09<00:00,  7.45s/it]

{'train_runtime': 17889.8727, 'train_samples_per_second': 3.119, 'train_steps_per_second': 0.134, 'train_loss': 3.1949695523579917, 'epoch': 89.0}


TrainOutput(global_step=2400, training_loss=3.1949695523579917, metrics={'train_runtime': 17889.8727, 'train_samples_per_second': 3.119, 'train_steps_per_second': 0.134, 'total_flos': 0.0, 'train_loss': 3.1949695523579917, 'epoch': 89.0})

In [19]:
model.save_pretrained("fine-tuned/models/chatPMB-pretrained-2")

# Get Tags by index

In [21]:
labels = pd.read_csv('data/dataset-question-v2.csv')
labels = labels.iloc[:-12]
labels = labels['tag']
labels

0         gr_hi
1         gr_ha
2         gr_pa
3         gr_si
4         gr_so
         ...   
541    beasiswa
542    beasiswa
543    beasiswa
544    beasiswa
545    beasiswa
Name: tag, Length: 546, dtype: object

# Usage of Pretrained Model

In [37]:
model = SentenceTransformer("fine-tuned/models/chatPMB-pretrained-2")

In [ ]:
sentence = model.encode("Berikan saya informasi seputar jalur masuk SNBT")
to_test = model.encode(df['sentence_2'])

In [ ]:
result = np.array(model.similarity(sentence, to_test))
index = np.argmax(result)
confidence = np.max(result)
detected_label = labels[index]
index, detected_label, result, confidence

(78,
 'jm_s1',
 array([[ 0.13337867,  0.29719597,  0.2966455 ,  0.2880215 ,  0.23856631,
          0.16570197,  0.33405572,  0.30675748,  0.36961758,  0.3351106 ,
          0.3716225 ,  0.37327492,  0.33403504,  0.44880426,  0.3478146 ,
          0.16618124,  0.15524584,  0.2868624 ,  0.27764806,  0.33482254,
          0.3877466 ,  0.3975867 ,  0.2877016 ,  0.30817634,  0.3870708 ,
          0.2539267 ,  0.40455842,  0.32194304,  0.36637944,  0.35080016,
          0.28612888,  0.32021773,  0.25793436,  0.27093685,  0.3716225 ,
          0.3478146 ,  0.42094195,  0.2868624 ,  0.36961758,  0.27647585,
          0.30668586,  0.3273719 ,  0.34952635,  0.27093685,  0.37501466,
          0.35411066,  0.4358852 ,  0.30603254,  0.3152501 ,  0.39809006,
          0.3453277 ,  0.411498  ,  0.36857313,  0.26992297,  0.2980612 ,
          0.3504343 ,  0.3009507 ,  0.5973962 ,  0.5125232 ,  0.62857425,
          0.37957358,  0.5355717 ,  0.42671186,  0.3279971 ,  0.60490924,
          0.48414862,  